In [7]:
import h5py
import numpy as np
from collections import defaultdict
from tqdm.auto import tqdm
from itertools import product

In [2]:
PATH = '../../datasets/usps/usps.h5'
with h5py.File(PATH, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [5]:
np.logspace(-5, 2, 10)

array([1.00000000e-05, 5.99484250e-05, 3.59381366e-04, 2.15443469e-03,
       1.29154967e-02, 7.74263683e-02, 4.64158883e-01, 2.78255940e+00,
       1.66810054e+01, 1.00000000e+02])

In [6]:
param_grid = {
    'T': [np.linspace(200, 50000, 10)],
    'l': [np.logspace(-5, 2, 10)]
}

In [37]:
values = param_grid.values()
list(values)

[[array([  200.        ,  5733.33333333, 11266.66666667, 16800.        ,
         22333.33333333, 27866.66666667, 33400.        , 38933.33333333,
         44466.66666667, 50000.        ])],
 [array([1.00000000e-05, 5.99484250e-05, 3.59381366e-04, 2.15443469e-03,
         1.29154967e-02, 7.74263683e-02, 4.64158883e-01, 2.78255940e+00,
         1.66810054e+01, 1.00000000e+02])]]

In [36]:
items = sorted(param_grid.items())
keys, values = zip(*items)
[dict(zip(keys, v)) for v in product(*values)]

[{'T': array([  200.        ,  5733.33333333, 11266.66666667, 16800.        ,
         22333.33333333, 27866.66666667, 33400.        , 38933.33333333,
         44466.66666667, 50000.        ]),
  'l': array([1.00000000e-05, 5.99484250e-05, 3.59381366e-04, 2.15443469e-03,
         1.29154967e-02, 7.74263683e-02, 4.64158883e-01, 2.78255940e+00,
         1.66810054e+01, 1.00000000e+02])}]

In [29]:
list(product(list(values)))

[(array([  200.        ,  5733.33333333, 11266.66666667, 16800.        ,
         22333.33333333, 27866.66666667, 33400.        , 38933.33333333,
         44466.66666667, 50000.        ]),
  array([1.00000000e-05, 5.99484250e-05, 3.59381366e-04, 2.15443469e-03,
         1.29154967e-02, 7.74263683e-02, 4.64158883e-01, 2.78255940e+00,
         1.66810054e+01, 1.00000000e+02]))]

In [13]:
list(product(param_grid))

[('T',), ('l',)]

In [65]:
d = {
    'a': 1,
    'b': 2
}

In [66]:
tuple(d.items())

(('a', 1), ('b', 2))

[{'a': 1, 'b': 4},
 {'a': 1, 'b': 5},
 {'a': 1, 'b': 6},
 {'a': 2, 'b': 4},
 {'a': 2, 'b': 5},
 {'a': 2, 'b': 6},
 {'a': 3, 'b': 4},
 {'a': 3, 'b': 5},
 {'a': 3, 'b': 6}]

In [52]:
items = d.items()
list(items)

[('a', [1, 2, 3]), ('b', [4, 5, 6])]

In [54]:
list(zip(*items))

[('a', 'b'), ([1, 2, 3], [4, 5, 6])]

In [50]:
list(product(*values))

[('b', 4), ('b', 5), ('b', 6)]